In [1]:
class Contours():
    def __init__(self, img, thresh, min_w, min_h, max_w=1000, max_h=1000):
        self.raw_img = img.copy()
        self.img = img.copy()
        self.thresh = thresh
        self.__min_w = min_w
        self.__min_h = min_h
        self.__max_w = max_w
        self.__max_h = max_h
        

    def find_contours(self):
        img = self.img
        # 二値化
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 画像を平滑化(白色雑音の除去). ksize:フィルタ窓枠の大きさ, sigmaX: ガウシアンの標準偏差値. 0だとカーネルのサイズから自動的に計算
        graygauss = cv2.GaussianBlur(gray, ksize=(3, 3), sigmaX=0)
        # 二値化. thresh:閾値.  maxval:閾値以上(指定により閾値以下のこともある)の値を持つ画素に対して割り当てられる値,
        # type:二値化の方法。cv2.THRESH_BINARY_INVだと白黒かつ反転
        # 返り値:[1]に画像が入っている
        im2 = cv2.threshold(graygauss, thresh = self.thresh, maxval=255,
                            type=cv2.THRESH_BINARY_INV)[1]

        # cv2.imshow("threshold", im2)
        # plt.imshow(im2, cmap="gray")
        # cv2.waitKey(0)
        # 輪郭抽出
        cnts = cv2.findContours(im2, mode=cv2.RETR_EXTERNAL,
                                method=cv2.CHAIN_APPROX_SIMPLE)[1]
        # 輪郭図示
        red = (0, 0, 255)
        size_list = []
        for pt in cnts:
            # 輪郭を含む長方形を作る
            x, y, w, h = cv2.boundingRect(pt)
            # 小さい領域はスルー
            if w < self.__min_w or h < self.__min_h:
                continue
            #大きいサイズもスルー
            if w > self.__max_w or h > self.__max_h:
                continue
            size_list.append((x, x + w, y, y+h))
            # 長方形を図示
            cv2.rectangle(img, (x, y), (x+w, y+h), red, thickness=10)
#         cv2.imshow("contours", img)
        self.size_list = size_list
        return img
    
    def get_rectangle_img(self):
        for lis in self.size_list:
            temp = self.raw_img[lis[2]:lis[3], lis[0]:lis[1]]
            yield temp

    
"""
    def show_cont(self):
        for lis in self.size_list:
            temp = self.raw_img[lis[2]:lis[3], lis[0]:lis[1]]
            yield temp

    def get_corner(self):
        for lis in self.size_list:
            temp = [[lis[0], lis[2]],
                    [lis[1], lis[2]],
                    [lis[1], lis[3]],
                    [lis[0], lis[3]],
                    ]
            yield temp

    # 領域の中心画素を返す
    def get_at(self):
        for lis in self.size_list:
            temp = [(lis[0] + lis[1]) * 0.5, (lis[2] + lis[3]) * 0.5]
            yield temp
            
    def get_diagonal_px(self):
        ret = []
        for lis in self.size_list:
            width = lis[1] - lis[0]
            height = lis[3] - lis[2]
            px = math.sqrt(width ** 2 + height ** 2)
            # print("px", px)
            ret.append(px)
        return ret

"""

'\n    def show_cont(self):\n        for lis in self.size_list:\n            temp = self.raw_img[lis[2]:lis[3], lis[0]:lis[1]]\n            yield temp\n\n    def get_corner(self):\n        for lis in self.size_list:\n            temp = [[lis[0], lis[2]],\n                    [lis[1], lis[2]],\n                    [lis[1], lis[3]],\n                    [lis[0], lis[3]],\n                    ]\n            yield temp\n\n    # 領域の中心画素を返す\n    def get_at(self):\n        for lis in self.size_list:\n            temp = [(lis[0] + lis[1]) * 0.5, (lis[2] + lis[3]) * 0.5]\n            yield temp\n            \n    def get_diagonal_px(self):\n        ret = []\n        for lis in self.size_list:\n            width = lis[1] - lis[0]\n            height = lis[3] - lis[2]\n            px = math.sqrt(width ** 2 + height ** 2)\n            # print("px", px)\n            ret.append(px)\n        return ret\n\n'

In [2]:
#スマホ画面抽出
import cv2

frame = cv2.imread("c.png")
# gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.imshow('frame',frame)
contours=Contours(frame, thresh = 125, min_w = 400, min_h = 400)
countor_img = contours.find_contours()
cv2.imshow("countor_img",countor_img)
for roi_img in contours.get_rectangle_img():
    cv2.imshow("roi", roi_img)
# cv2.imwrite("a2.png",countor_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [42]:
cv2.imshow('frame',roi_img)
contours=Contours(roi_img, thresh = 125, min_w = 50, min_h = 50, )
countor_img = contours.find_contours()
cv2.imshow("countor_img",countor_img)
# for roi_img in contours.get_rectangle_img():
#     cv2.imshow("roi", roi_img)
# cv2.imwrite("a2.png",countor_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [38]:
#カメラで写真撮影
import cv2

cap = cv2.VideoCapture(2)
cap.set(3, 1280)
cap.set(4, 720)

gray = []
while True:
    _, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('frame',gray)
    # contours=Contours(frame, 125)
    # countor_img = contours.find_contours()
    cv2.imshow("countor_img",frame)
#     cv2.imshow("gray",gray)
    key = cv2.waitKey(10)
    if key == ord("q"):
        break
    elif key == ord("p"):
        cv2.imwrite("c.png", frame)
        cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()

In [31]:
cv2.imshow("countor_img",roi_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

13

In [32]:
cv2.destroyAllWindows()
cap.release()

In [1]:
"""
Path tracking simulation with pure pursuit steering and PID speed control.
author: Atsushi Sakai (@Atsushi_twi)
        Guillaume Jacquenot (@Gjacquenot)
"""
import numpy as np
import math
import matplotlib.pyplot as plt

# Parameters
k = 0.1  # look forward gain
Lfc = 2.0  # [m] look-ahead distance
Kp = 1.0  # speed proportional gain
dt = 0.1  # [s] time tick
WB = 2.9  # [m] wheel base of vehicle

show_animation = True


class State:

    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.rear_x = self.x - ((WB / 2) * math.cos(self.yaw))
        self.rear_y = self.y - ((WB / 2) * math.sin(self.yaw))

    def update(self, a, delta):
        self.x += self.v * math.cos(self.yaw) * dt
        self.y += self.v * math.sin(self.yaw) * dt
        self.yaw += self.v / WB * math.tan(delta) * dt
        self.v += a * dt
        self.rear_x = self.x - ((WB / 2) * math.cos(self.yaw))
        self.rear_y = self.y - ((WB / 2) * math.sin(self.yaw))

    def calc_distance(self, point_x, point_y):
        dx = self.rear_x - point_x
        dy = self.rear_y - point_y
        return math.hypot(dx, dy)


class States:

    def __init__(self):
        self.x = []
        self.y = []
        self.yaw = []
        self.v = []
        self.t = []

    def append(self, t, state):
        self.x.append(state.x)
        self.y.append(state.y)
        self.yaw.append(state.yaw)
        self.v.append(state.v)
        self.t.append(t)


def proportional_control(target, current):
    a = Kp * (target - current)

    return a


class TargetCourse:

    def __init__(self, cx, cy):
        self.cx = cx
        self.cy = cy
        self.old_nearest_point_index = None

    def search_target_index(self, state):

        # To speed up nearest point search, doing it at only first time.
        if self.old_nearest_point_index is None:
            # search nearest point index
            dx = [state.rear_x - icx for icx in self.cx]
            dy = [state.rear_y - icy for icy in self.cy]
            d = np.hypot(dx, dy)
            ind = np.argmin(d)
            self.old_nearest_point_index = ind
        else:
            ind = self.old_nearest_point_index
            distance_this_index = state.calc_distance(self.cx[ind],
                                                      self.cy[ind])
            while True:
                distance_next_index = state.calc_distance(self.cx[ind + 1],
                                                          self.cy[ind + 1])
                if distance_this_index < distance_next_index:
                    break
                ind = ind + 1 if (ind + 1) < len(self.cx) else ind
                distance_this_index = distance_next_index
            self.old_nearest_point_index = ind

        Lf = k * state.v + Lfc  # update look ahead distance

        # search look ahead target point index
        while Lf > state.calc_distance(self.cx[ind], self.cy[ind]):
            if (ind + 1) >= len(self.cx):
                break  # not exceed goal
            ind += 1

        return ind, Lf


def pure_pursuit_steer_control(state, trajectory, pind):
    ind, Lf = trajectory.search_target_index(state)

    if pind >= ind:
        ind = pind

    if ind < len(trajectory.cx):
        tx = trajectory.cx[ind]
        ty = trajectory.cy[ind]
    else:  # toward goal
        tx = trajectory.cx[-1]
        ty = trajectory.cy[-1]
        ind = len(trajectory.cx) - 1

    alpha = math.atan2(ty - state.rear_y, tx - state.rear_x) - state.yaw

    delta = math.atan2(2.0 * WB * math.sin(alpha) / Lf, 1.0)

    return delta, ind


def plot_arrow(x, y, yaw, length=1.0, width=0.5, fc="r", ec="k"):
    """
    Plot arrow
    """

    if not isinstance(x, float):
        for ix, iy, iyaw in zip(x, y, yaw):
            plot_arrow(ix, iy, iyaw)
    else:
        plt.arrow(x, y, length * math.cos(yaw), length * math.sin(yaw),
                  fc=fc, ec=ec, head_width=width, head_length=width)
        plt.plot(x, y)


def main():
    #  target course
    cx = np.arange(0, 50, 0.5)
    cy = [math.sin(ix / 5.0) * ix / 2.0 for ix in cx]

    target_speed = 10.0 / 3.6  # [m/s]

    T = 100.0  # max simulation time

    # initial state
    state = State(x=-0.0, y=-3.0, yaw=0.0, v=0.0)

    lastIndex = len(cx) - 1
    time = 0.0
    states = States()
    states.append(time, state)
    target_course = TargetCourse(cx, cy)
    target_ind, _ = target_course.search_target_index(state)

    while T >= time and lastIndex > target_ind:

        # Calc control input
        ai = proportional_control(target_speed, state.v)
        di, target_ind = pure_pursuit_steer_control(
            state, target_course, target_ind)

        state.update(ai, di)  # Control vehicle

        time += dt
        states.append(time, state)

        if show_animation:  # pragma: no cover
            plt.cla()
            # for stopping simulation with the esc key.
            plt.gcf().canvas.mpl_connect(
                'key_release_event',
                lambda event: [exit(0) if event.key == 'escape' else None])
            plot_arrow(state.x, state.y, state.yaw)
            plt.plot(cx, cy, "-r", label="course")
            plt.plot(states.x, states.y, "-b", label="trajectory")
            plt.plot(cx[target_ind], cy[target_ind], "xg", label="target")
            plt.axis("equal")
            plt.grid(True)
            plt.title("Speed[km/h]:" + str(state.v * 3.6)[:4])
            plt.pause(0.001)

    # Test
    assert lastIndex >= target_ind, "Cannot goal"

    if show_animation:  # pragma: no cover
        plt.cla()
        plt.plot(cx, cy, ".r", label="course")
        plt.plot(states.x, states.y, "-b", label="trajectory")
        plt.legend()
        plt.xlabel("x[m]")
        plt.ylabel("y[m]")
        plt.axis("equal")
        plt.grid(True)

        plt.subplots(1)
        plt.plot(states.t, [iv * 3.6 for iv in states.v], "-r")
        plt.xlabel("Time[s]")
        plt.ylabel("Speed[km/h]")
        plt.grid(True)
        plt.show()


if __name__ == '__main__':
    print("Pure pursuit path tracking simulation start")
    main()

Pure pursuit path tracking simulation start


<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

...

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>